In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_local



In [8]:
def order_points(pts):
    rect=np.zeros((4,2),dtype="float32")
    pts_sum=np.sum(pts,axis=1)
    rect[0]=pts[np.argmin(pts_sum)]
    rect[2]=pts[np.argmax(pts_sum)]
    
    pts_diff=np.diff(pts,axis=1)
    rect[1]=pts[np.argmin(pts_diff)]
    rect[3]=pts[np.argmax(pts_diff)]
    
    return rect

In [9]:
def four_point_transform(image,pts):
    rect=order_points(pts)
    tl,tr,br,bl=rect
    
    widthA=np.sqrt((tl[0]-tr[0])**2+(tl[1]-tr[1])**2)
    widthB=np.sqrt((bl[0]-br[0])**2+(bl[1]-br[1])**2)
    
    width=max(int(widthA),int(widthB))
    
    heightA=np.sqrt((tl[0]-bl[0])**2+(tl[1]-bl[1])**2)
    heightB=np.sqrt((tr[0]-br[0])**2+(br[1]-tr[1])**2)
    
    height=max(int(heightA),int(heightB))
    
    dst = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (width, height))
    
    return warped



In [10]:
cap=cv2.VideoCapture(0)

while True:
    
    ret,img=cap.read()
    ratio=img.shape[0]/500
    orig=img.copy()
    warped=img.copy()
    
    
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)

    edged=cv2.Canny(blur,75,200)
    cnts,_=cv2.findContours(edged,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    contour = None
    maxArea = 0
    
    for c in cnts:
        area = cv2.contourArea(c)
        
        if area > 25000:
            peri = cv2.arcLength(c, True)
            polygon = cv2.approxPolyDP(c, 0.02*peri, True)
            
            if area>maxArea and len(polygon)==4:
                contour = polygon
                maxArea = area
    
    if contour is not None:
        cv2.drawContours(img,[contour],-1,(0,255,0),2)
        warped = four_point_transform(img, contour.reshape(4, 2))
        warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        T = threshold_local(warped, 11, offset = 10, method = "gaussian")
        warped = (warped > T).astype("uint8") * 255
        cv2.imshow("result",warped)
        
    
    cv2.imshow("Original", img)
    
    if cv2.waitKey(30) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
    
    